In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,11079
2,Huelva,Confirmados PDIA 14 días,820
3,Huelva,Tasa PDIA 14 días,"160,55041380890194"
4,Huelva,Confirmados PDIA 7 días,355
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,3
629,Municipio de Huelva sin especificar,Total Confirmados,127
630,Municipio de Huelva sin especificar,Curados,106


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  11079.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  3130.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1355 personas en los últimos 7 días 

Un positivo PCR cada 601 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,11079.0,355.0,820.0,510743.0,69.506582,160.550414,26.0
Huelva-Costa,6254.0,242.0,533.0,288115.0,83.994238,184.995575,14.0
Condado-Campiña,3413.0,107.0,259.0,155057.0,69.006881,167.035348,9.0
Huelva (capital),3130.0,106.0,239.0,143663.0,73.783786,166.361554,7.0
Cartaya,414.0,34.0,66.0,19974.0,170.221288,330.429558,3.0
Isla Cristina,448.0,39.0,73.0,21264.0,183.408578,343.303236,2.0
Palos de la Frontera,251.0,21.0,36.0,11289.0,186.021791,318.894499,2.0
Palma del Condado (La),540.0,20.0,60.0,10761.0,185.856333,557.568999,1.0
Trigueros,134.0,3.0,6.0,7713.0,38.895371,77.790743,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,104.0,10.0,21.0,3371.0,296.647879,622.960546,0.0
Palma del Condado (La),540.0,20.0,60.0,10761.0,185.856333,557.568999,1.0
Bonares,65.0,17.0,22.0,6058.0,280.620667,363.156157,1.0
Isla Cristina,448.0,39.0,73.0,21264.0,183.408578,343.303236,2.0
Cartaya,414.0,34.0,66.0,19974.0,170.221288,330.429558,3.0
Palos de la Frontera,251.0,21.0,36.0,11289.0,186.021791,318.894499,2.0
Escacena del Campo,97.0,1.0,6.0,2284.0,43.782837,262.697023,0.0
Aljaraque,438.0,19.0,49.0,21260.0,89.369708,230.479774,0.0
Lepe,965.0,25.0,63.0,27431.0,91.137764,229.667165,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Bollullos Par del Condado,249.0,1.0,15.0,14272.0,7.006726,105.100897,-1.0,0.066667
San Juan del Puerto,155.0,2.0,17.0,9300.0,21.505376,182.795699,1.0,0.117647
Sierra de Huelva-Andévalo Central,1286.0,3.0,23.0,67571.0,4.439774,34.038271,0.0,0.130435
Escacena del Campo,97.0,1.0,6.0,2284.0,43.782837,262.697023,0.0,0.166667
Paterna del Campo,68.0,2.0,8.0,3509.0,56.996295,227.985181,0.0,0.250000
Moguer,379.0,13.0,41.0,22088.0,58.855487,185.621152,1.0,0.317073
Punta Umbría,235.0,3.0,9.0,15242.0,19.682456,59.047369,0.0,0.333333
Palma del Condado (La),540.0,20.0,60.0,10761.0,185.856333,557.568999,1.0,0.333333
Gibraleón,350.0,1.0,3.0,12607.0,7.932101,23.796304,0.0,0.333333
